# Outlier Detection v1


In [1]:
import numpy as np

import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

In [2]:
#pandas is a powerful Python data analysis toolkit
#fast, flexible, and expressive data structures designed to make 
#working with relational or labeled data
df = pd.read_json("coronaextrausa0.json")
df.head()
#After testing the source many are posted by advertising agencies

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,NaN,2016-11-25 22:05:05,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,16,False,NaN,802271920146128896,802271920146128896,...,NaN,NaN,NaN,12,False,NaN,"<a href=""http://percolate.com"" rel=""nofollow"">...",Que comience la temporada de compartir momento...,False,"{'follow_request_sent': False, 'has_extended_p..."
1,NaN,NaN,2016-11-25 22:00:08,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,38,False,NaN,802270673183731712,802270673183731712,...,NaN,NaN,NaN,22,False,NaN,"<a href=""http://percolate.com"" rel=""nofollow"">...",Find your tree yet? https://t.co/C7VCHj2e9b,False,"{'follow_request_sent': False, 'has_extended_p..."
2,NaN,NaN,2016-11-20 16:06:14,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,13,False,NaN,800369672776466432,800369672776466432,...,NaN,NaN,NaN,5,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Cuando te dejas llevar por la música ocurren c...,False,"{'follow_request_sent': False, 'has_extended_p..."
3,NaN,NaN,2016-11-20 16:05:43,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,29,False,NaN,800369540844679168,800369540844679168,...,NaN,NaN,NaN,8,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",That moment when your song takes you to that h...,False,"{'follow_request_sent': False, 'has_extended_p..."
4,NaN,NaN,2016-11-11 21:22:18,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,15,False,NaN,797187721127149568,797187721127149568,...,NaN,NaN,NaN,9,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#FightNight done right. Catch Kovalev vs Ward ...,False,"{'follow_request_sent': False, 'has_extended_p..."


In [3]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'extended_entities', 'favorite_count', 'favorited', 'geo', 'id',
       'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted',
       'retweeted_status', 'source', 'text', 'truncated', 'user'],
      dtype='object')

In [4]:
df.dtypes

contributors                        float64
coordinates                         float64
created_at                   datetime64[ns]
entities                             object
extended_entities                    object
favorite_count                        int64
favorited                              bool
geo                                 float64
id                                    int64
id_str                                int64
in_reply_to_screen_name              object
in_reply_to_status_id               float64
in_reply_to_status_id_str           float64
in_reply_to_user_id                 float64
in_reply_to_user_id_str             float64
is_quote_status                        bool
lang                                 object
place                               float64
possibly_sensitive                  float64
quoted_status                        object
quoted_status_id                    float64
quoted_status_id_str                float64
retweet_count                   

In [5]:
ndf = df[['text', 'created_at', 'retweet_count']]

ndf.head(5)

,text,created_at,retweet_count
0,Que comience la temporada de compartir momento...,2016-11-25 22:05:05,12
1,Find your tree yet? https://t.co/C7VCHj2e9b,2016-11-25 22:00:08,22
2,Cuando te dejas llevar por la música ocurren c...,2016-11-20 16:06:14,5
3,That moment when your song takes you to that h...,2016-11-20 16:05:43,8
4,#FightNight done right. Catch Kovalev vs Ward ...,2016-11-11 21:22:18,9


In [6]:
df2 = pd.read_json("coronaextrausa1.json")
ndf2 = df2[['text', 'created_at', 'retweet_count']]
ndf2.head(5)

,text,created_at,retweet_count
0,@ktuck_91 Safe travels on your journey. Don't ...,2016-07-08 21:56:48,0
1,@quetallitabusca Cómo debe de ser. Salud!,2016-07-07 15:35:00,0
2,@adriana02022879 En el mar la vida es más sabr...,2016-07-07 15:30:08,0
3,@JoshTaerk And #BastilleDay coming up next wee...,2016-07-05 20:59:12,1
4,@the1andonlypaco Permission granted.,2016-07-05 20:58:51,1


In [7]:
df3 = pd.read_json("coronaextrausa2.json")
ndf3 = df3[['text', 'created_at', 'retweet_count']]
ndf3.head(5)

,text,created_at,retweet_count
0,@ToddyFur @chalkybear13 Salud!,2016-04-18 20:19:38,0
1,@chamberzone_ Here's to you. Salud!,2016-04-18 20:18:16,0
2,"@jaredcooper4 More beach vibes, please!",2016-04-18 20:18:01,0
3,@solita_valencia Gotta file some limes. #TaxDay,2016-04-18 20:17:36,0
4,@KodiboyLaurie That summer feeling.,2016-04-18 20:17:16,1


In [8]:
df4 = pd.read_json("coronaextrausa3.json")
ndf4 = df4[['text', 'created_at', 'retweet_count']]
ndf4.head(5)

,text,created_at,retweet_count
0,¡Pero que vista!\nMuéstranos tu #ViewFromMyCo...,2016-01-27 17:36:42,7
1,Change the forecast of your day with #CoronaEx...,2016-01-25 04:10:50,10
2,Cualquier día del año es perfecto para disfrut...,2016-01-25 01:58:26,9
3,Dooooown - Set - Hut - Hut - #CoronaExtra. htt...,2016-01-22 19:00:03,13
4,¡Ya estamos listos para el Gran Juego! #Corona...,2016-01-22 17:00:05,18


In [9]:
df5 = pd.read_json("coronaextrausa4.json")
ndf5 = df5[['text', 'created_at', 'retweet_count']]
ndf5.head(5)

,text,created_at,retweet_count
0,CoronaRita &gt; margarita. \nFave this to shak...,2015-07-23 21:04:52,11
1,Un amor de verano que no quieres que termine. ...,2015-07-21 23:04:34,11
2,This is one summer fling we never want to end....,2015-07-21 21:08:30,26
3,Receta para el verano perfecto:\nRelájate.\nRe...,2015-07-18 07:23:45,13
4,Summer vacation recipe: \nRelax. \nRecenter. \...,2015-07-17 21:00:05,13


In [10]:
df6 = pd.read_json("coronaextrausa5.json")
ndf6 = df6[['text', 'created_at', 'retweet_count']]
ndf6.head(5)

,text,created_at,retweet_count
0,@Buzzfeed’s 11 reasons why summer is better f...,2014-05-28 22:30:05,13
1,Show us how you enjoy your #CoronaSummer. / E...,2014-05-27 20:00:11,20
2,There’s always room for something Extra at the...,2014-05-26 20:00:09,27
3,Long weekend essentials. The only thing missin...,2014-05-23 22:30:06,35
4,How are you keeping cool this #CoronaSummer? ¿...,2014-05-22 20:00:13,7


In [11]:
df7 = pd.read_json("coronaextrausa6.json")
ndf7 = df7[['text', 'created_at', 'retweet_count']]
ndf7.head(5)

,text,created_at,retweet_count
0,Retweet if you think this should be a universa...,2013-05-07 22:38:49,45
1,Share your Corona Summer pictures &amp; one mi...,2013-05-06 20:58:54,3
2,"Happy Corona de Mayo! Friends, food, fun and C...",2013-05-06 03:32:29,19
3,Get your Cinco de Mayo weekend started with fu...,2013-05-03 22:36:29,26
4,Cinco de Mayo is almost here! Here’s something...,2013-05-03 18:31:37,10


In [12]:
cdf = pd.concat([ndf,ndf2], ignore_index=True)
cdf2 = pd.concat([cdf,ndf3], ignore_index=True)
cdf3 = pd.concat([cdf2,ndf4], ignore_index=True)
cdf4 = pd.concat([cdf3,ndf5], ignore_index=True)
cdf5 = pd.concat([cdf4,ndf6], ignore_index=True)
corona_map_reduce = pd.concat([cdf5,ndf7], ignore_index=True)
corona_map_reduce

,text,created_at,retweet_count
0,Que comience la temporada de compartir momento...,2016-11-25 22:05:05,12
1,Find your tree yet? https://t.co/C7VCHj2e9b,2016-11-25 22:00:08,22
2,Cuando te dejas llevar por la música ocurren c...,2016-11-20 16:06:14,5
3,That moment when your song takes you to that h...,2016-11-20 16:05:43,8
4,#FightNight done right. Catch Kovalev vs Ward ...,2016-11-11 21:22:18,9
5,Kovalev vs. Ward este 19 de Noviembre. Agarra ...,2016-11-11 21:21:26,2
6,Feliz Día de los Muertos. Hoy celebremos con a...,2016-11-01 14:55:45,15
7,"Happy Día de los Muertos, to you and all who v...",2016-11-01 14:54:14,9
8,"Este Día de Muertos, asegúrate de tener sufici...",2016-11-01 00:27:36,9
9,"This Day of the Dead, be sure to offer enough ...",2016-10-31 23:19:12,26
